In [1]:
!pip install escnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.9/373.9 kB 10.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 72.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00
  Created wheel for py3nj: filename=py3nj-0.2.1-cp310-cp310-linux_x86_64.whl size=44764 sha256=76df1d02a853ffd71ecf73a600613eb4a1e0dbe82e4b18d12d4015b0ca72ef19
  Stored in directory: /root/.cache/pip/wheels/71/e9/70/30a34ed6dbc8b54ce93f25c091be4cf7a24319e27d953a882b
Successfully built py3nj


In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from escnn.gspaces import flipRot2dOnR2
from escnn.nn import FieldType, R2Conv, NormBatchNorm, ReLU, SequentialModule, EquivariantModule, FieldType, GeometricTensor, PointwiseNonLinearity, InnerBatchNorm
from PIL import Image, ImageOps
from skimage.util import random_noise
from skimage import exposure
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score

`Helper functions`

In [14]:
def load_image(image_path, target_size):
    image = Image.open(image_path).convert('RGB' if target_size[2] == 3 else 'L')
    transform = transforms.Compose([
        transforms.Resize(target_size[:2]),
    ])
    return transform(image)

def load_real_data(data_dir, target_size=(256, 256)):
    landslide_dir = os.path.join(data_dir, 'landslide')
    non_landslide_dir = os.path.join(data_dir, 'non-landslide')

    images = []
    dems = []
    labels = []

    def process_image(image_path, dem_path, label):
        image = load_image(image_path, target_size + (3,))
        dem = load_image(dem_path, target_size + (1,))

        images.append(transforms.ToTensor()(image))
        dems.append(transforms.ToTensor()(dem))
        labels.append(label)

    for filename in os.listdir(os.path.join(landslide_dir, 'image')):
        if filename.endswith(".png"):
            image_path = os.path.join(landslide_dir, 'image', filename)
            dem_path = os.path.join(landslide_dir, 'dem', filename)
            process_image(image_path, dem_path, label=1)  # Label 1 for landslide

    for filename in os.listdir(os.path.join(non_landslide_dir, 'image')):
        if filename.endswith(".png"):
            image_path = os.path.join(non_landslide_dir, 'image', filename)
            dem_path = os.path.join(non_landslide_dir, 'dem', filename)
            process_image(image_path, dem_path, label=0)  # Label 0 for non-landslide

    images = torch.stack(images)
    dems = torch.stack(dems)
    labels = torch.tensor(labels, dtype=torch.float32)

    return images, dems, labels


`Network components`

In [7]:
class ConvBlock(EquivariantModule):
    def __init__(self, space, in_channels, out_channels):
        super(ConvBlock, self).__init__()

        # Define input and output types
        self.space = space
        group_size = len(self.gspace.fibergroup.elements)
        
        self.in_type = FieldType(self.space, [self.space.regular_repr] * (in_channels // group_size))
        self.out_type = FieldType(self.space, [self.space.regular_repr] * (in_channels // group_size))

        self.conv = SequentialModule(
            R2Conv(self.in_type, self.out_type, kernel_size=3, padding=1, bias=False),
            InnerBatchNorm(self.out_type),
            ReLU(self.out_type),
            R2Conv(self.out_type, self.out_type, kernel_size=3, padding=1, bias=False),
            InnerBatchNorm(self.out_type),
            ReLU(self.out_type)
        )

    def forward(self, x):
        if not isinstance(x, GeometricTensor):
            geo_x = GeometricTensor(x, self.in_type)
        else:
            geo_x = x
        return self.conv(geo_x).tensor

    def evaluate_output_shape(self, input_shape):
        return self.conv.evaluate_output_shape(input_shape)

    # def export(self):
    #     raise NotImplementedError("Exporting to standard PyTorch not implemented yet")

class Encoder(EquivariantModule):
    def __init__(self):
        super(Encoder, self).__init__()

        # Define the G-space for P4m equivariance
        self.space = flipRot2dOnR2(4)  # 4 rotations + flips

        self.conv1_rgb = ConvBlock(self.space, 3, 16)
        self.conv1_dem = ConvBlock(self.space, 1, 8)
        self.conv2 = ConvBlock(self.space, 24, 32)
        self.conv3 = ConvBlock(self.space, 32, 64)
        self.conv4 = ConvBlock(self.space, 64, 128)
        self.bottleneck = ConvBlock(self.space, 128, 256)

        self.global_pool = nn.AdaptiveAvgPool2d(1)

        # Fully connected layer without equivariance
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x_rgb, x_dem):
        # x_rgb = GeometricTensor(x_rgb, self.conv1_rgb.in_type)
        # x_dem = GeometricTensor(x_dem, self.conv1_dem.in_type)

        c1_rgb = self.conv1_rgb(x_rgb)
        c1_dem = self.conv1_dem(x_dem)

        combined = torch.cat([c1_rgb.tensor, c1_dem.tensor], dim=1)
        # combined = GeometricTensor(combined, self.conv2.in_type)

        c2 = self.conv2(combined)
        p2 = nn.MaxPool2d(kernel_size=2)(c2.tensor)
        # p2 = GeometricTensor(p2, self.conv3.in_type)

        c3 = self.conv3(p2)
        p3 = nn.MaxPool2d(kernel_size=2)(c3.tensor)
        # p3 = GeometricTensor(p3, self.conv4.in_type)

        c4 = self.conv4(p3)
        p4 = nn.MaxPool2d(kernel_size=2)(c4.tensor)
        # p4 = GeometricTensor(p4, self.bottleneck.in_type)

        bn = self.bottleneck(p4)
        pooled = self.global_pool(bn.tensor)

        return self.fc(pooled)

    def evaluate_output_shape(self, input_shape):
        raise NotImplementedError("Shape evaluation not implemented yet")

    # def export(self):
    #     raise NotImplementedError("Exporting to standard PyTorch not implemented yet")

# Usage example
encoder = Encoder()
x_rgb = torch.randn(1, 3, 256, 256)  # Input for RGB
x_dem = torch.randn(1, 1, 256, 256)  # Input for DEM
output = encoder(x_rgb, x_dem)
print(output)

AssertionError: Error! The size of the tensor torch.Size([1, 3, 256, 256]) does not match the size of the field type 24.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
from torchinfo import summary

model = Encoder().to(device)
summary(model, input_size=[(1, 3, 256, 256), (1, 1, 256, 256)]) 

In [ ]:
data_dir = '/kaggle/input/bijie/Bijie_dataset'
images, dems, labels = load_real_data(data_dir, target_size=(256, 256))

In [ ]:
len(dems)

In [ ]:
from sklearn.model_selection import train_test_split

X_train_img, X_test_img, X_train_dem, X_test_dem, y_train, y_test = train_test_split(
    images, dems, labels, test_size=0.2, random_state=42
)

print(X_train_img.shape, X_train_dem[0].shape, y_train[0].shape)
print("Training data size:", len(X_train_img))
print("Testing data size:", len(X_test_img))

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, images, dems, masks):
        self.images = images
        self.dems = dems
        self.masks = masks

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return {
            'image': self.images[idx],
            'dem': self.dems[idx],
            'label': self.masks[idx]
        }

train_dataset = CustomDataset(X_train_img, X_train_dem, y_train)
val_dataset = CustomDataset(X_test_img, X_test_dem, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
model = Encoder().to(device)
epochs = 200
best_val_loss = float('inf')
best_model_path = 'best_unet_model.pth'

In [ ]:
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for batch in train_loader:
        images = batch['image'].to(device)
        dems = batch['dem'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(images, dems).squeeze(1) 

        binary_preds = (outputs > 0.5).float()

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0.0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch in val_loader:
            images = batch['image'].to(device)
            dems = batch['dem'].to(device)
            labels = batch['label'].to(device)

            outputs = model(images, dems).squeeze(1)

            binary_preds = (outputs > 0.5).float()

            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = binary_preds.cpu().numpy().astype(int)
            targets = labels.cpu().numpy().astype(int)
            all_preds.append(preds)
            all_targets.append(targets)

    val_loss /= len(val_loader)

    all_preds = np.concatenate([pred.flatten() for pred in all_preds])
    all_targets = np.concatenate([target.flatten() for target in all_targets])

    accuracy = accuracy_score(all_targets, all_preds)
    precision = precision_score(all_targets, all_preds, zero_division=0)
    recall = recall_score(all_targets, all_preds, zero_division=0)
    f1 = f1_score(all_targets, all_preds)
    iou = jaccard_score(all_targets, all_preds)

    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss}, Val Loss: {val_loss}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, IoU: {iou}, F1: {f1}")

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"Saved best model with val loss: {best_val_loss}")

print("Training complete. Best model saved to", best_model_path)